In [1]:
from collections import defaultdict

# Is a triangulated surface orientable?

A triangulated surface will be represented as a graph. Each node in the graph represents a triangle. Two nodes in the graph are connected when corresponding triangles share a common edge.

Note: when a triangulation represents a surface without boundary, then graph is cubic.

In [2]:
class Triangle:
    """
    A triangle is represented as a list of its
    vertices (labeled with natural numbers).
    """
    def __init__(self, vertices):
        assert len(vertices) == 3, 'A triangle should have 3 vertices'
        self.vertices = sorted(vertices)
        
    @property
    def edges(self):
        """
        Return the list of edges 'in the positive direction'.
        """
        return (
                (self.vertices[0], self.vertices[1]),
                (self.vertices[1], self.vertices[2]),
                (self.vertices[2], self.vertices[0]),
               )

    def contains_edge(self, edge):
        '''
        Return True, if the triangle contains a given edge. 
        The edge is given as a tuple.
        '''
        return edge[0] in self.vertices and edge[1] in self.vertices
    
    def __repr__(self):
        return str(self.vertices)

In [3]:
class Node:
    """
    A node in a graph.
    Each node has pointers to its neighbours and pointer to the data the node contains.
    """
    def __init__(self, data=None):
        self.neighbours = set()
        self.data = data

List of triangles in a triangulation of a torus and a projective plane.

In [4]:
torus_triangles = [
    (0, 1, 6), (0, 4, 6), (1, 6, 7), (1, 2, 7), (2, 7, 4), (2, 0, 4), (4, 5, 8), (4, 6, 8), (6, 3, 8),
    (6, 7, 3), (3, 7, 5), (7, 4, 5), (0, 1, 5), (1, 5, 8), (1, 2, 8), (2, 8, 3), (2, 0, 3), (0, 5, 3),
]

projective_plane_triangles = [
    (1, 2, 7), (1, 7, 3), (2, 7, 5), (7, 3, 6), (2, 3, 5), (4, 5, 7),
    (7, 4, 6), (2, 6, 3), (4, 5, 3), (4, 6, 2), (1, 3, 4), (4, 1, 2),
]

Create triangulations from the set of triangles.

In [5]:
def create_triangulation(triangles):
    triangles = [Triangle(triangle) for triangle in triangles]
    n = max([max(triangle.vertices) for triangle in triangles]) + 1
    nodes = [Node(triangle) for triangle in triangles]
    # For each vertex compute the set of nodes representing triangles containing the vertex.    
    vertex_nodes = defaultdict(set)
    for node in nodes:
        for vertex in node.data.vertices:
            vertex_nodes[vertex].add(node)
    # Make connection between neighbourhood nodes.
    for node in nodes:
        for (v1, v2) in node.data.edges:
            neighbour = (vertex_nodes[v1] & vertex_nodes[v2]) - set([node])
            assert len(neighbour) == 1
            node.neighbours.add(neighbour.pop())
        assert len(node.neighbours) == 3
    return nodes    

Create triangulations from the set of triangles for a torus and a projective plane.

In [6]:
torus_triangulation = create_triangulation(torus_triangles)
projective_plane_triangulation = create_triangulation(projective_plane_triangles)
assert len(torus_triangles) == len(torus_triangles)
assert len(projective_plane_triangulation) == len(projective_plane_triangles)

Helper function which construct the order structure for a given triangulation.

In [7]:
def get_orientation(edge, edges):
    """
    Returt the orientation of edge in the given set of edges.
    It can be 0, 1 or 2 (edge is one of the edges)
    or 4, 5 or 6 (edge with a positeva orientation is one of the edges).
    """
    try:
        index =  edges.index(edge)
    except Exception:
        index = edges.index((edge[1], edge[0])) + 4
    return index

def sym(orientation):
    """
    Get the orientation with the same leading edge in the oposite direction.
    """
    return (orientation + 4) % 8

def enext(orientation):
    """
    Get the next orientation of the same type.
    """
    if orientation <= 2:
        return (orientation + 1) % 3
    return ((orientation + 1) % 3) + 4

    
def order_triangulation(triangulation):
    """
    Construct ordered triangles for a given triangulation.
    """
    for node in triangulation:
        triangle = node.data
        neighbours = node.neighbours
        # Two assumptions made here:
        # - triangle vertices are ordered
        # - function edges of triangle returns edges  
        #   in the positive orientation
        ordered = dict()
        for i, edge in enumerate(triangle.edges):
            # Get the neighbour that shares the common edge with our triangle
            neighbour = [n for n in neighbours if n.data.contains_edge(edge)][0]
            orientation = get_orientation(edge, neighbour.data.edges)
            ordered[i] = (neighbour.data, orientation)
            ordered[sym(i)] = (neighbour.data, sym(orientation))
        triangle.ordered = ordered

Finally: a function which checks whether a given triangulation is orientable.

In [8]:
def orientable(triangles):
    '''
    Is a triangulation orientable?
    '''
    triangulation = create_triangulation(triangles)
    order_triangulation(triangulation)
    return is_orientable(triangulation[0].data.ordered[0])

def same_orientation_type(o1, o2):
    '''
    Are two orientations of the same type.
    '''
    assert o1 in [0, 1, 2, 4, 5, 6]
    assert o2 in [0, 1, 2, 4, 5, 6]
    return o1 >> 2 == o2 >> 2

def is_orientable(oriented_triangle):
    (triangle, orientation) = oriented_triangle
    chosen_orientation = getattr(triangle, 'orientation', None)
    if chosen_orientation is None:
        triangle.orientation = orientation
        (t1, o1) = triangle.ordered[sym(orientation)]
        (t2, o2) = triangle.ordered[enext(sym(orientation))]
        (t3, o3) = triangle.ordered[enext(enext(sym(orientation)))]
        return is_orientable((t1, o1)) and is_orientable((t2, o2)) and is_orientable((t3, o3))
    else:
        return same_orientation_type(orientation, chosen_orientation)

In [9]:
orientable(torus_triangles)

True

In [10]:
orientable(projective_plane_triangles)

False